In [ ]:
import matplotlib.pyplot as plt
import os
os.environ.update(dict(CUDA_VISIBLE_DEVICES='3'))

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
data = torch.load('/ssd1/tta/imagenet_val_resnet50_lyrfts_full.pth')
# data = torch.load('/ssd1/tta/inc4_resnet50_shf_bn_full.pth')

In [ ]:
#features, labels
features_inc = data['features']
features_im = data['ifeatures'] # 50000, C (2048, 1024, 512, 256)
labels = data['labels']
logits_inc = data['logits']
logits_im = data['ilogits']
correct = data['correct'].type(torch.int)

layer = -1

def cidx(i):
    return labels == i